In [ ]:
# ⚽ İngiliz Premier Ligi Veri Analizi (EDA)
# Berke Özkul - Football Match Score Prediction Project

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Grafik ayarları
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("🚀 Veri analizi başlıyor...")
print("=" * 50)


In [ ]:
# 1. Veri Yükleme ve İlk İnceleme
import os
import glob

# Tüm CSV dosyalarını listele
data_path = "../data/"
csv_files = glob.glob(os.path.join(data_path, "*.csv"))
print(f"Toplam {len(csv_files)} CSV dosyası bulundu:")
for file in sorted(csv_files):
    print(f"  - {os.path.basename(file)}")

# İlk dosyayı inceleyelim
first_season = pd.read_csv(csv_files[0])
print(f"\n📊 İlk sezon veri boyutu: {first_season.shape}")
print(f"📊 Sütun sayısı: {len(first_season.columns)}")
print(f"📊 Satır sayısı: {len(first_season)}")

print("\n🏆 İlk 5 sütun:")
print(first_season.columns[:5].tolist())

print("\n📈 Veri tipi dağılımı:")
print(first_season.dtypes.value_counts())


In [ ]:
# 2. Tek Sezon Detaylı İnceleme
print("🔍 İlk sezon detaylı analizi:")
print("=" * 40)

# Temel istatistikler
print(f"📅 Tarih aralığı: {first_season['Date'].min()} - {first_season['Date'].max()}")
print(f"🏟️  Benzersiz ev sahibi takım sayısı: {first_season['HomeTeam'].nunique()}")
print(f"🏃 Benzersiz deplasman takımı sayısı: {first_season['AwayTeam'].nunique()}")

# Takım listesi
print(f"\n⚽ Takımlar ({first_season['HomeTeam'].nunique()} takım):")
teams = sorted(first_season['HomeTeam'].unique())
for i, team in enumerate(teams, 1):
    print(f"{i:2}. {team}")

# Hedef değişkenler analizi
print(f"\n🥅 Gol dağılımı:")
print(f"  Ev sahibi ortalama gol: {first_season['FTHG'].mean():.2f}")
print(f"  Deplasman ortalama gol: {first_season['FTAG'].mean():.2f}")
print(f"  Toplam maç başına gol: {(first_season['FTHG'] + first_season['FTAG']).mean():.2f}")

# Sonuç dağılımı
result_counts = first_season['FTR'].value_counts()
result_pct = first_season['FTR'].value_counts(normalize=True) * 100
print(f"\n🏆 Sonuç dağılımı:")
for result in ['H', 'D', 'A']:
    label = {'H': 'Ev sahibi galibiyeti', 'D': 'Beraberlik', 'A': 'Deplasman galibiyeti'}[result]
    print(f"  {label}: {result_counts[result]} ({result_pct[result]:.1f}%)")


In [ ]:
# 3. Tüm Sezonları Birleştirme
print("🔄 Tüm sezonları birleştiriyoruz...")

all_seasons = []
for file_path in sorted(csv_files):
    # Sezon adını dosya adından çıkar
    season_name = os.path.basename(file_path).replace('.csv', '').replace('E0 ', '')
    
    # CSV'yi oku
    df = pd.read_csv(file_path)
    
    # Sezon sütunu ekle
    df['Season'] = season_name
    
    # Tarih sütununu düzelt
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')
    
    all_seasons.append(df)
    print(f"  ✅ {season_name}: {len(df)} maç")

# Birleştir
combined_data = pd.concat(all_seasons, ignore_index=True)
print(f"\n🎯 Toplam birleştirilmiş veri:")
print(f"  📊 Boyut: {combined_data.shape}")
print(f"  📅 Tarih aralığı: {combined_data['Date'].min().strftime('%Y-%m-%d')} - {combined_data['Date'].max().strftime('%Y-%m-%d')}")
print(f"  🏆 Toplam sezon: {combined_data['Season'].nunique()}")
print(f"  ⚽ Toplam maç: {len(combined_data)}")

# Sezon başına maç sayısı
print(f"\n📈 Sezon başına maç sayısı:")
season_counts = combined_data['Season'].value_counts().sort_index()
for season, count in season_counts.items():
    print(f"  {season}: {count} maç")


In [ ]:
# 4. Eksik Veri Analizi
print("🔍 Eksik veri analizi:")
print("=" * 30)

# Genel eksik veri oranı
total_cells = combined_data.shape[0] * combined_data.shape[1]
missing_cells = combined_data.isnull().sum().sum()
print(f"📊 Toplam hücre sayısı: {total_cells:,}")
print(f"❌ Eksik veri sayısı: {missing_cells:,}")
print(f"📈 Eksik veri oranı: {(missing_cells/total_cells)*100:.2f}%")

# Sütun bazında eksik veri
print(f"\n🔍 Sütun bazında eksik veri (sadece eksik olanlar):")
missing_by_column = combined_data.isnull().sum()
missing_columns = missing_by_column[missing_by_column > 0].sort_values(ascending=False)

if len(missing_columns) > 0:
    for col, missing_count in missing_columns.head(10).items():
        missing_pct = (missing_count / len(combined_data)) * 100
        print(f"  {col}: {missing_count:,} ({missing_pct:.1f}%)")
else:
    print("  🎉 Hiç eksik veri yok!")

# Kritik sütunların eksik veri kontrolü
critical_columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
print(f"\n🎯 Kritik sütunlar eksik veri kontrolü:")
for col in critical_columns:
    missing = combined_data[col].isnull().sum()
    if missing > 0:
        print(f"  ❌ {col}: {missing} eksik")
    else:
        print(f"  ✅ {col}: Tam")


In [ ]:
# 5. Görselleştirmeler
print("📊 Veri görselleştirmeleri oluşturuluyor...")

# 5.1 Gol dağılımı histogramı
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Ev sahibi goller
axes[0,0].hist(combined_data['FTHG'], bins=range(0, 8), alpha=0.7, color='blue', edgecolor='black')
axes[0,0].set_title('Ev Sahibi Gol Dağılımı', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Gol Sayısı')
axes[0,0].set_ylabel('Frekans')
axes[0,0].grid(True, alpha=0.3)

# Deplasman goller
axes[0,1].hist(combined_data['FTAG'], bins=range(0, 8), alpha=0.7, color='red', edgecolor='black')
axes[0,1].set_title('Deplasman Gol Dağılımı', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Gol Sayısı')
axes[0,1].set_ylabel('Frekans')
axes[0,1].grid(True, alpha=0.3)

# Sonuç dağılımı
result_counts = combined_data['FTR'].value_counts()
labels = ['Ev Sahibi\nGalibiyeti', 'Beraberlik', 'Deplasman\nGalibiyeti']
colors = ['green', 'orange', 'red']
axes[1,0].pie(result_counts, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
axes[1,0].set_title('Maç Sonuçları Dağılımı', fontsize=14, fontweight='bold')

# Toplam gol dağılımı
total_goals = combined_data['FTHG'] + combined_data['FTAG']
axes[1,1].hist(total_goals, bins=range(0, 12), alpha=0.7, color='purple', edgecolor='black')
axes[1,1].set_title('Toplam Gol Dağılımı (Maç Başına)', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Toplam Gol')
axes[1,1].set_ylabel('Frekans')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# İstatistikler
print(f"\n📈 Gol İstatistikleri:")
print(f"  🏠 Ev sahibi ortalama: {combined_data['FTHG'].mean():.2f}")
print(f"  🏃 Deplasman ortalama: {combined_data['FTAG'].mean():.2f}")
print(f"  ⚽ Maç başına ortalama toplam gol: {total_goals.mean():.2f}")
print(f"  📊 En yüksek ev sahibi gol: {combined_data['FTHG'].max()}")
print(f"  📊 En yüksek deplasman gol: {combined_data['FTAG'].max()}")


In [ ]:
# 6. Takım Performans Analizi
print("🏆 Takım performanslarını analiz ediyoruz...")

# En çok gol atan takımlar (ev sahibi olarak)
home_goals = combined_data.groupby('HomeTeam')['FTHG'].agg(['mean', 'sum', 'count']).round(2)
home_goals.columns = ['Ortalama_Gol', 'Toplam_Gol', 'Maç_Sayısı']
home_goals = home_goals.sort_values('Ortalama_Gol', ascending=False)

print("🏠 Ev sahibi olarak en çok gol atan takımlar (ilk 10):")
print(home_goals.head(10))

# En çok gol atan takımlar (deplasman olarak)
away_goals = combined_data.groupby('AwayTeam')['FTAG'].agg(['mean', 'sum', 'count']).round(2)
away_goals.columns = ['Ortalama_Gol', 'Toplam_Gol', 'Maç_Sayısı']
away_goals = away_goals.sort_values('Ortalama_Gol', ascending=False)

print(f"\n🏃 Deplasman olarak en çok gol atan takımlar (ilk 10):")
print(away_goals.head(10))

# Galibiyetler analizi
home_wins = combined_data[combined_data['FTR'] == 'H']['HomeTeam'].value_counts()
away_wins = combined_data[combined_data['FTR'] == 'A']['AwayTeam'].value_counts()

print(f"\n🎯 En çok ev sahibi galibiyeti alan takımlar (ilk 5):")
for i, (team, wins) in enumerate(home_wins.head().items(), 1):
    print(f"  {i}. {team}: {wins} galibiyet")

print(f"\n🎯 En çok deplasman galibiyeti alan takımlar (ilk 5):")
for i, (team, wins) in enumerate(away_wins.head().items(), 1):
    print(f"  {i}. {team}: {wins} galibiyet")


In [ ]:
# 7. Bahis Oranları Analizi
print("💰 Bahis oranlarını analiz ediyoruz...")

# Bahis oranları sütunlarını kontrol et
betting_columns = [col for col in combined_data.columns if 'B365' in col]
print(f"📊 Bet365 sütunları: {betting_columns}")

if betting_columns:
    # Null olmayan bahis verisi olanları filtrele
    betting_data = combined_data[betting_columns + ['FTR']].dropna()
    print(f"📈 Bahis verisi olan maç sayısı: {len(betting_data)}")
    
    # Bahis oranlarının temel istatistikleri
    print(f"\n💰 Bahis Oranları İstatistikleri:")
    print(betting_data[betting_columns].describe().round(2))
    
    # Favoriler vs sonuçlar analizi
    if 'B365H' in betting_data.columns and 'B365A' in betting_data.columns:
        # En düşük oran = favori takım
        betting_data_copy = betting_data.copy()
        betting_data_copy['Favori'] = 'D'  # Default beraberlik
        
        # Ev sahibi favori
        home_fav_mask = (betting_data_copy['B365H'] < betting_data_copy['B365A']) & (betting_data_copy['B365H'] < betting_data_copy['B365D'])
        betting_data_copy.loc[home_fav_mask, 'Favori'] = 'H'
        
        # Deplasman favori  
        away_fav_mask = (betting_data_copy['B365A'] < betting_data_copy['B365H']) & (betting_data_copy['B365A'] < betting_data_copy['B365D'])
        betting_data_copy.loc[away_fav_mask, 'Favori'] = 'A'
        
        # Favori takımların doğru tahmin oranı
        correct_predictions = (betting_data_copy['Favori'] == betting_data_copy['FTR']).sum()
        total_predictions = len(betting_data_copy)
        accuracy = (correct_predictions / total_predictions) * 100
        
        print(f"\n🎯 Bahis Favorileri Analizi:")
        print(f"  📊 Toplam maç (bahis verisi olan): {total_predictions}")
        print(f"  ✅ Doğru tahmin: {correct_predictions}")
        print(f"  📈 Doğruluk oranı: {accuracy:.1f}%")
        
        # Favori dağılımı
        fav_dist = betting_data_copy['Favori'].value_counts()
        print(f"\n🏆 Favori Dağılımı:")
        for fav_type, count in fav_dist.items():
            label = {'H': 'Ev sahibi', 'D': 'Beraberlik', 'A': 'Deplasman'}[fav_type]
            pct = (count / total_predictions) * 100
            print(f"  {label}: {count} ({pct:.1f}%)")
else:
    print("❌ Bahis oranları bulunamadı!")


In [ ]:
# 8. Veriyi Kaydetme
print("💾 Temizlenmiş veriyi kaydediyoruz...")

# Temizlenmiş veriyi CSV olarak kaydet
output_path = "../data/combined_data.csv"
combined_data.to_csv(output_path, index=False)
print(f"✅ Veri kaydedildi: {output_path}")

# Özet rapor
print(f"\n📋 VERI ANALİZİ ÖZET RAPORU:")
print("=" * 50)
print(f"📊 Toplam veri boyutu: {combined_data.shape[0]:,} satır × {combined_data.shape[1]} sütun")
print(f"📅 Tarih aralığı: {combined_data['Date'].min().strftime('%Y-%m-%d')} - {combined_data['Date'].max().strftime('%Y-%m-%d')}")
print(f"🏆 Sezon sayısı: {combined_data['Season'].nunique()}")
print(f"⚽ Benzersiz takım sayısı: {combined_data['HomeTeam'].nunique()}")
print(f"🥅 Ortalama ev sahibi gol: {combined_data['FTHG'].mean():.2f}")
print(f"🏃 Ortalama deplasman gol: {combined_data['FTAG'].mean():.2f}")
print(f"📈 Ev sahibi galibiyet oranı: {(combined_data['FTR']=='H').mean()*100:.1f}%")
print(f"🤝 Beraberlik oranı: {(combined_data['FTR']=='D').mean()*100:.1f}%")
print(f"✈️  Deplasman galibiyet oranı: {(combined_data['FTR']=='A').mean()*100:.1f}%")

missing_pct = (combined_data.isnull().sum().sum() / (combined_data.shape[0] * combined_data.shape[1])) * 100
print(f"❌ Toplam eksik veri oranı: {missing_pct:.2f}%")

print(f"\n🎯 Sonraki Adımlar:")
print("  1. ✅ Veri keşfi ve analizi tamamlandı")
print("  2. 🔄 Veri ön işleme (data_preprocessing.py)")
print("  3. ⚙️  Özellik mühendisliği")
print("  4. 🤖 Model eğitimi")
print("  5. 🚀 API geliştirme")
print("  6. 📱 Flutter uygulaması")

print(f"\n✨ Veri analizi başarıyla tamamlandı! ✨")
